In [12]:
import os
import codecs
import jieba
import re

from sklearn.utils import shuffle

In [13]:
category = ['星座', '股票', '房产', '时尚', '体育', '社会', '家居', '游戏', '彩票', '科技', '教育', '时政', '娱乐', '财经']

# 每篇文档保留的文档数量
#per_class_max_docs = 1000

def load_data_to_mini(path, to_path, per_class_max_docs=1000):
    """
    处理清华大学语料库，将类别和文档处理成fasttext 所需要的格式
    :param path: 
    :param to_path: 
    :return: 
    """
    # 抽取后的语料库
    corpus = []
    if not os.path.isdir(path):
        print('path error')
    # 列举当前目录下的所有子列别目录
    with codecs.open(to_path, 'w+',encoding='utf-8',errors='ignore') as f:
        for files in os.listdir(path):
            curr_path = os.path.join(path, files)
            print(curr_path)
            if os.path.isdir(curr_path):
                count = 0
                docs = []
                for file in os.listdir(curr_path):
                    count += 1
                    if count > per_class_max_docs:
                        break
                    file_path = os.path.join(curr_path, file)
                    
                    # 将取出来的文本存入文件中
                    with codecs.open(file_path, 'r', encoding='utf-8',errors='ignore') as fd:
                        docs.append('__label__' + files + ' ' + ' '.join(jieba.cut(re.sub('[  \n\r\t]+', '', fd.readline()))))
                        s = ' '.join(jieba.cut(re.sub('[  \n\r\t]+', '', fd.read())))
                        f.write('__label__' + files + ' ' + s+'\n')
            corpus.append(docs)
            
    return corpus


corpus = load_data_to_mini('../data/THUCNews', 'thu_data_all.txt', 3000)

../data/THUCNews\体育
../data/THUCNews\娱乐
../data/THUCNews\家居
../data/THUCNews\彩票
../data/THUCNews\房产
../data/THUCNews\教育
../data/THUCNews\时尚
../data/THUCNews\时政
../data/THUCNews\星座
../data/THUCNews\游戏
../data/THUCNews\社会
../data/THUCNews\科技
../data/THUCNews\股票
../data/THUCNews\财经


In [14]:
print('corpus size(%d,%d)' %(len(corpus), len(corpus[0])))
corpus[0][1]

corpus size(14,3000)


'__label__体育 商瑞华 首战 复仇 心切 中国 玫瑰 要 用 美国 方式 攻克 瑞典'

In [15]:
def split_data_with_label(corpus):
    """
    将数据划分为训练数据和样本标签
    :param corpus: 
    :return: 
    """
    input_x = []
    input_y = []

    tag = []
    if os.path.isfile(corpus):
        with codecs.open(corpus, 'rb',encoding='utf-8') as f:
            for line in f:
                tag.append(line)

    else:
        for docs in corpus:
            for doc in docs:
                tag.append(doc)
    tag = shuffle(tag)
    for doc in tag:
#         print(doc)
        index = doc.find(' ')
        input_y.append(doc[:index])
        input_x.append(doc[index + 1 :])

    # 打乱数据，避免在采样的时候出现类别不均衡现象
    # datasets = np.column_stack([input_x, input_y])
    # np.random.shuffle(datasets)
    # input_x = []
    # input_y = []
    # for i in datasets:
    #     input_x.append(i[:-1])
    #     input_y.append(i[-1:])
    return [input_x, input_y]

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics.scorer import make_scorer
from sklearn import linear_model
from sklearn import metrics

from time import time

def feature_extractor(input_x, case='tfidf', max_df=1.0, min_df=0.0):
    """
    特征抽取
    :param corpus: 
    :param case: 不同的特征抽取方法
    :return: 
    """
    return TfidfVectorizer(token_pattern='\w', ngram_range=(1,2), max_df=max_df, min_df=min_df).fit_transform(input_x)

# 拆分数据集
def split_data_to_train_and_test(corpus, indices=0.2, random_state=10, shuffle=True):
    """
    将数据划分为训练数据和测试数据
    :param corpus: [input_x]
    :param indices: 划分比例
    :random_state: 随机种子
    :param shuffle: 是否打乱数据
    :return: 
    """
    input_x, y = corpus

    # 切分数据集
    x_train, x_dev, y_train, y_dev = train_test_split(input_x, y, test_size=indices, random_state=10)
    print("Vocabulary Size: {:d}".format(input_x.shape[1]))
    print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))
    return x_train, x_dev, y_train, y_dev

### 模型训练

In [17]:
# 加载语料
corpus = split_data_with_label('thu_data_all.txt')

input_x, y = corpus
# 特征选择
input_x = feature_extractor(input_x, 'tfidf')
# 划分训练数据和测试数据
train_x, test_x, train_y, test_y = split_data_to_train_and_test([input_x, y])

# 训练以及测试
t0 = time()


from sklearn.naive_bayes import MultinomialNB  # 导入多项式贝叶斯算法  
  
# 训练分类器：输入词袋向量和分类标签，alpha:0.001 alpha越小，迭代次数越多，精度越高  
# clf = MultinomialNB(alpha=0.1).fit(train_set.tdm, train_set.label)  
  
######################################################  
 
from sklearn.ensemble import RandomForestClassifier 
print ('*************************随机森林分类器***********************' ) 
 
clf = RandomForestClassifier(oob_score=True, random_state=10)   
clf.fit(train_x, train_y)  
  
# 预测分类结果  
 
print ('*************************开始预测************************'  )
predicted = clf.predict(test_x)  


 
for flabel,expct_cate in zip(test_y,predicted):  
    if flabel != expct_cate:  
        print (": 实际类别:",flabel," -->预测类别:",expct_cate ) 
        
print("预测完毕!!!" ) 
  
# 计算分类精度：  
from sklearn import metrics  
def metrics_result(actual, predict):  
    print ('精度:{0:.3f}'.format(metrics.precision_score(actual, predict,average='weighted'))  )
    print ('召回:{0:0.3f}'.format(metrics.recall_score(actual, predict,average='weighted')) ) 
    print ('f1-score:{0:.3f}'.format(metrics.f1_score(actual, predict,average='weighted')) ) 
    
metrics_result(test_y, predicted)  



Vocabulary Size: 1476958
Train/Dev split: 33600/8401
*************************随机森林分类器***********************


E:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
E:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:458: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
E:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:463: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


*************************开始预测************************
: 实际类别: __label__彩票  -->预测类别: __label__财经
: 实际类别: __label__教育  -->预测类别: __label__社会
: 实际类别: __label__房产  -->预测类别: __label__家居
: 实际类别: __label__财经  -->预测类别: __label__家居
: 实际类别: __label__时尚  -->预测类别: __label__教育
: 实际类别: __label__教育  -->预测类别: __label__房产
: 实际类别: __label__时政  -->预测类别: __label__房产
: 实际类别: __label__家居  -->预测类别: __label__时尚
: 实际类别: __label__财经  -->预测类别: __label__体育
: 实际类别: __label__科技  -->预测类别: __label__教育
: 实际类别: __label__科技  -->预测类别: __label__房产
: 实际类别: __label__社会  -->预测类别: __label__时尚
: 实际类别: __label__时政  -->预测类别: __label__娱乐
: 实际类别: __label__股票  -->预测类别: __label__时政
: 实际类别: __label__科技  -->预测类别: __label__房产
: 实际类别: __label__游戏  -->预测类别: __label__教育
: 实际类别: __label__财经  -->预测类别: __label__房产
: 实际类别: __label__财经  -->预测类别: __label__时政
: 实际类别: __label__星座  -->预测类别: __label__娱乐
: 实际类别: __label__体育  -->预测类别: __label__社会
: 实际类别: __label__教育  -->预测类别: __label__时政
: 实际类别: __label__社会  -->预测类别: __label__家居
: 实际类别: __label__娱乐  -

: 实际类别: __label__时尚  -->预测类别: __label__家居
: 实际类别: __label__财经  -->预测类别: __label__时尚
: 实际类别: __label__股票  -->预测类别: __label__社会
: 实际类别: __label__财经  -->预测类别: __label__教育
: 实际类别: __label__科技  -->预测类别: __label__家居
: 实际类别: __label__教育  -->预测类别: __label__游戏
: 实际类别: __label__时政  -->预测类别: __label__星座
: 实际类别: __label__娱乐  -->预测类别: __label__社会
: 实际类别: __label__科技  -->预测类别: __label__财经
: 实际类别: __label__财经  -->预测类别: __label__星座
: 实际类别: __label__星座  -->预测类别: __label__家居
: 实际类别: __label__娱乐  -->预测类别: __label__教育
: 实际类别: __label__教育  -->预测类别: __label__游戏
: 实际类别: __label__游戏  -->预测类别: __label__教育
: 实际类别: __label__社会  -->预测类别: __label__时政
: 实际类别: __label__科技  -->预测类别: __label__房产
: 实际类别: __label__教育  -->预测类别: __label__娱乐
: 实际类别: __label__时政  -->预测类别: __label__家居
: 实际类别: __label__教育  -->预测类别: __label__家居
: 实际类别: __label__教育  -->预测类别: __label__体育
: 实际类别: __label__教育  -->预测类别: __label__体育
: 实际类别: __label__家居  -->预测类别: __label__时尚
: 实际类别: __label__股票  -->预测类别: __label__时政
: 实际类别: __label__科技  -->预测类别: __la

: 实际类别: __label__财经  -->预测类别: __label__股票
: 实际类别: __label__时尚  -->预测类别: __label__家居
: 实际类别: __label__星座  -->预测类别: __label__家居
: 实际类别: __label__财经  -->预测类别: __label__股票
: 实际类别: __label__时政  -->预测类别: __label__科技
: 实际类别: __label__股票  -->预测类别: __label__财经
: 实际类别: __label__科技  -->预测类别: __label__房产
: 实际类别: __label__财经  -->预测类别: __label__股票
: 实际类别: __label__教育  -->预测类别: __label__时政
: 实际类别: __label__游戏  -->预测类别: __label__教育
: 实际类别: __label__游戏  -->预测类别: __label__教育
: 实际类别: __label__股票  -->预测类别: __label__教育
: 实际类别: __label__娱乐  -->预测类别: __label__体育
: 实际类别: __label__教育  -->预测类别: __label__星座
: 实际类别: __label__科技  -->预测类别: __label__教育
: 实际类别: __label__财经  -->预测类别: __label__社会
: 实际类别: __label__社会  -->预测类别: __label__时政
: 实际类别: __label__时尚  -->预测类别: __label__家居
: 实际类别: __label__科技  -->预测类别: __label__时政
: 实际类别: __label__时政  -->预测类别: __label__游戏
: 实际类别: __label__星座  -->预测类别: __label__体育
: 实际类别: __label__时尚  -->预测类别: __label__家居
: 实际类别: __label__教育  -->预测类别: __label__娱乐
: 实际类别: __label__社会  -->预测类别: __la

E:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
E:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
